In [1]:
import re
import sqlite3
from urllib.request import urlopen
from html import unescape

In [2]:
def fetch(url: str) -> str:
    """引数で与えられたURLのWebページを取得
    
    Argument:
        url(str): WebページのURL
    Returns:
        str: Webページ情報
    """
    with urlopen(url) as f:
        encoding = f.info().get_content_charset(failobj="utf-8")
        html = f.read().decode(encoding)
        return html

In [8]:
from typing import List

def scrape(html: str) -> List[dict]:
    """引数で与えられたHTMLから正規表現で書籍の情報を抜き出す
    
    Arguments:
        html(str): HTML
    Returns:
        List[dict]: Webページ情報
    """              
    books = []
    for partial_html in re.findall(r'<a itemprop="url".*?</ul>\s*</a></li>', html, re.DOTALL):
        url = re.search(r'<a itemprop="url" href="(.*?)">', partial_html).group(1)
        title = re.search(r'<p itemprop="name".*?</p>', partial_html).group(0)
        title = re.sub(r'<.*?>', '', title) # タグを取り除く
        title = unescape(title) # 文字参照を元に戻す
        books.append({"url": url, "title": title})
    
    return books

In [10]:
def save(db_path: str, books: List[dict]) -> None:
    """書籍リストをSQLiteに保存する
    
    Arguments:
        db_path(str): SQLiteファイルパス
        books(List[dict]): Webページ情報リスト
    """
    with sqlite3.connect(db_path) as conn:
        c = conn.cursor()
        c.execute('DROP TABLE IF EXISTS books')
        c.execute('''
            CREATE TABLE books (title text, url text)
        ''')
        c.executemany('INSERT INTO books VALUES (:title, :url)', books)
        conn.commit()

In [11]:
html = fetch("http://sample.scraping-book.com/dp")
books = scrape(html)
save('books.db', books)